# Validation and Test Functions of `PDEsolver`

In [1]:
# Import all necessary modules
import PDEsolver as pde
import numpy as np
from scipy.integrate import solve_ivp

## Validations - examples of invalid input (exception handling)

In [2]:
t = pde.time_domain(-10,0.1)

Value Error: `tmax` and `dt` must be numeric values greater than 0.


In [3]:
t = pde.time_domain('a',10)

Value Error: `tmax` and `dt` must be numeric values greater than 0.


In [4]:
t = pde.time_domain(10,11)

Value Error: `dt` must be greater than or equal to `tmax`.


In [5]:
grid = pde.create_grid(-10,128)

Value Error: `length` and `num_of_points` must be numeric values greater than 0.


In [6]:
grid = pde.create_grid('a','b')

Value Error: `length` and `num_of_points` must be numeric values greater than 0.


In [7]:
g = pde.create_grid(10,128)
f = np.exp(g.x)
der = pde.derivative(g,f,-2)

Value Error: `order` must be a numeric value greater than or equal to 0.


In [8]:
t_bur = pde.time_domain(10,0.1)
grid_bur = pde.create_grid(32,512)
x_bur = grid_bur.x
D_bur = 0.01                        # Extreme choice of diffusivity
u0_bur = -np.sin(2*np.pi*x_bur/32)
u_bur = solve_ivp(pde.burger_eq,[t_bur[0],t_bur[-1]],u0_bur,args=(grid_bur,D_bur),t_eval = t_bur,method = 'DOP853')
pde.plot_anim(t_bur,grid_bur,u0_bur,u_bur)  # Exception handling of the plotting function plot_anim()

Sorry, there is something wrong with the parameter value or initial condition selection.


## Tests

In [9]:
# Test function for the class create_grid()
def test_grid(tol):
    '''
    Test using x = [-2pi,-3pi/2,-pi,-pi/2,0,pi/2,pi,3pi/2]
    True wavenum = [0,0.5,1,1.5,-2,-1.5,-1,-0.5]
    '''
    l = 4*np.pi
    n = 8
    grid = pde.create_grid(l,n)
    
    true_x = np.array([-2*np.pi,-3*np.pi/2,-np.pi,-np.pi/2,0,np.pi/2,np.pi,3*np.pi/2])
    calc_x = grid.x
    test_phys_domain = np.abs(true_x - calc_x).mean() < tol
    
    true_wavenum = 2*np.pi*np.array([0,1,2,3,-4,-3,-2,-1])/(8*np.pi/2)
    calc_wavenum = grid.wavenum
    test_four_domain = np.abs(true_wavenum - calc_wavenum).mean() < tol
    
    return (test_phys_domain and test_four_domain).all()

t = 1e-12
assert test_grid(t)

In [10]:
# Test functions for the function derivative()
def test_derivative(tol):
    '''
    Test using f = sin(2x) for x in -2pi to 2pi, find the second derivative
    True answer = -4sin(2x), which is -4*f
    '''
    grid = pde.create_grid(4*np.pi,100)
    x = grid.x
    f = np.sin(2*x)
    true_df = -4*f
    calc_df = pde.derivative(grid,f,2)    
    return np.abs(calc_df - true_df).mean() < tol

t = 1e-12
assert test_derivative(t)

In [11]:
# Test functions for solving the PDEs with Gussian initial condition
def gaussian_solution(t, x, σ, D): 
    '''
    This is a function that returns the Gaussian solution at time = t, domain = x, initial width of the curve = σ, and diffusivity = D.
    '''
    return σ / np.sqrt(σ**2 + 2*D*t) * np.exp(-x**2 / (2*(σ**2 + 2*D*t)))

def analytical_sol(x,t,D,v):
    '''
    This function returns a numpy array that contains the analytical solution of the Gaussian function with advection and diffusion based on the advection-diffusion equation. 
    The dimension of the matrix is len(x) * len(t), which means each column of the matrix is the solution at time = t.
    '''
    analytical_sol = np.zeros((len(x),len(t)))
    for i in range(len(t)):
        analytical_sol[:,i] = gaussian_solution(t[i],x - v*t[i],0.8,D)
    return analytical_sol

In [12]:
# Test functions for solving the PDEs with Gussian initial condition
def test_solve_dudt(D,v,tol):
    '''
    Use the Gaussian Equation to test the solutions by comparing the analytical solution with the numerical solution.
    Absolute mean error between the final states of both analytical and numerical solutions are calculated.
    '''
    grid = pde.create_grid(10*np.pi,512)
    x = grid.x
    t = pde.time_domain(1,0.025)    # Time domain
    
    u0 = gaussian_solution(t[0],x,0.8,D)

    
    # Numerical solution for advection equation
    numerical_sol  = solve_ivp(pde.adv_diff_eq,[t[0],t[-1]],u0,args=(grid,D,v),t_eval=t,method='DOP853')
    numerical_sol_f = numerical_sol.y[:,-1]
    
    # Analytical solution for advection equation: u(x,t) = ui(x - v*t)
    ana_sol = analytical_sol(x,t,D,v)
    analytical_sol_f = ana_sol[:,-1]

    # # If you wanted to plot the final stages of the numerical and analytical solutions, please remove the # below.
    # plt.plot(x,numerical_sol_f, label='Numerical solution')
    # plt.plot(x,analytical_sol_f, label='Analytical solution')
    # plt.plot(x,numerical_sol_f - analytical_sol_f,label='error')
    # plt.legend()
    # plt.show()
    
    mean_error = np.mean(np.abs(numerical_sol_f - analytical_sol_f))
    
    print('Absolute mean error in the final state equals to {}.'.format(mean_error))
    
    return mean_error < tol

t = 1e-6
# Case 1: Constant advection only, no diffusion
assert test_solve_dudt(0,4,t)
# Case 2: Constant diffusion only, no advection
assert test_solve_dudt(4,0,t)
print('All tests passsed.')

Absolute mean error in the final state equals to 1.8009414754093208e-07.
Absolute mean error in the final state equals to 4.99171905140956e-08.
All tests passsed.
